In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import os
import pandas as pd

In [ ]:
%matplotlib qt
#Battery voltage data taken on 7/8/22. described in lab book

timeArrFull = [0,1,1.5,2,2.5,3,4,5,8,10,15,20,25,30, 40, 50, 60, 75, 90, 120, 145, 180, 240]

vArrFull = [14.44,14.38,14.34,14.32,14.30,14.28,14.24,14.20,
        14.11,14.05,13.94,13.86,13.79, 13.74, 13.65, 13.57,
       13.52, 13.46, 13.41, 13.34, 13.30, 13.28, 13.25]

timeArr = [0,20, 40,  60, 75, 90, 120, 145, 180, 240]

vArr = [14.44, 13.86, 13.65,
       13.52, 13.46, 13.41, 13.34, 13.30, 13.28, 13.25]

plt.plot(timeArrFull, vArrFull)
plt.scatter(timeArrFull, vArrFull)
plt.xlabel('Time (minutes)')
plt.ylabel('Volts')
plt.title('Battery Voltage vs Time')


In [ ]:
%matplotlib inline
#fit and plot

def voltageExp(t, V0, tau, b):
    V = V0 * np.exp(-t/tau) + b
    return V


popt, cov = opt.curve_fit(voltageExp, timeArrFull[:], vArrFull[:], p0=[1, 30, 13])
print('fitting function: \nV = V0 * np.exp(-t/tau) + b\n')

print('fit parameters:')
print('V0 =', round(popt[0], 3), '+-', round(cov[0, 0], 5), 'Volts')
print('tau =', round(popt[1], 3),'+-', round(cov[1, 1], 5), 'Minutes')
print('b =', round(popt[2],3),'+-', round(cov[2, 2], 5), 'Volts')

timeInterp = np.linspace(0, max(timeArr), 1000)
vInterp = voltageExp(timeInterp, popt[0], popt[1], popt[2])

plt.plot(timeArr, vArr)
plt.scatter(timeArrFull, vArrFull, label = 'data')
plt.plot(timeInterp, vInterp, label = 'fit')
plt.xlabel('Time (minutes)')
plt.ylabel('Volts')
plt.legend();

In [ ]:
#try fitting less data

for i in range(-15, 0):
    print('exclude last', i, 'data points')
    popt, cov = opt.curve_fit(voltageExp, timeArrFull[:i], vArrFull[:i], p0=[1,30,13])
    timeInterp = np.linspace(0, max(timeArr), 1000)
    vInterp = voltageExp(timeInterp, popt[0], popt[1], popt[2])
    print('fit parameters:')
    print('V0 =', round(popt[0], 3), '+-', round(cov[0, 0], 5), 'Volts')
    print('tau =', round(popt[1], 3),'+-', round(cov[1, 1], 5), 'Minutes')
    print('b =', round(popt[2],3),'+-', round(cov[2, 2], 5), 'Volts')
    print()
    plt.plot(timeInterp, vInterp, label = round(i))
    
print('all data')
popt, cov = opt.curve_fit(voltageExp, timeArr, vArr, p0=[1,30,13])
timeInterp = np.linspace(0, max(timeArr), 1000)
vInterp = voltageExp(timeInterp, popt[0], popt[1], popt[2])
print('fit parameters:')
print('V0 =', round(popt[0], 3), '+-', round(cov[0, 0], 5), 'Volts')
print('tau =', round(popt[1], 3),'+-', round(cov[1, 1], 5), 'Minutes')
print('b =', round(popt[2],3),'+-', round(cov[2, 2], 5), 'Volts')
print()
plt.plot(timeInterp, vInterp, label = 'all data')

plt.scatter(timeArr, vArr)
plt.xlabel('Time (minutes)')
plt.ylabel('Volts')
plt.legend()
plt.title('Voltage vs time curve fit excluding last data points')

In [ ]:
#import gain data taken 7/7/22 and load into df

#calculate frequency to include. 
#10k points from 20-320MHz, 10k pts/300MHz = 33.3 pts/MHz
f1 = 20 #MHz
f2 = 320 #MHz
pts = 1e4
ptsPerMHz = pts / (f2 - f1)  #pts/MHz


startFreq = 45 #MHz
stopFreq = 305 #MHz


startIdx = int((startFreq - f1) * ptsPerMHz)
stopIdx = int(pts - (f2 - stopFreq) * ptsPerMHz)

#build list of file names

# folder path
dir_path = r'./data'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
res.sort()

#init and pack DF
df_gainData = pd.DataFrame()
df_gainData['freqs'] = np.load(dir_path + '/' + 'freqArr.npy')[startIdx:stopIdx]
freqs = (np.load(dir_path + '/' + 'freqArr.npy')/1e6)[startIdx:stopIdx]

for file in res:
    if file == 'freqArr.npy':
        continue
    v = file[0:5]
    if file[26:28] == 'Of':
        s = 'Off'
    else:
        s = 'On'
    col = v + 'V_switch' + s
    
    df_gainData[col] = (np.load(dir_path + '/' + file))[startIdx:stopIdx]
    
df_gainData.head()

In [ ]:
%matplotlib qt
plt.plot(freqs, df_gainData['13.00V_switchOff'], label = '13V')
plt.plot(freqs, df_gainData['13.30V_switchOff'], label = '13.3V')
plt.plot(freqs, df_gainData['13.50V_switchOff'], label = '13.5V')
plt.plot(freqs, df_gainData['14.00V_switchOff'], label = '14V')
plt.plot(freqs, df_gainData['14.20V_switchOff'], label = '14.2V')
plt.plot(freqs, df_gainData['14.40V_switchOff'], label = '14.4V')
plt.plot(freqs, df_gainData['14.55V_switchOff'], label = '14.55V')

plt.title('Switch Off')
plt.legend()
plt.xlabel('Frequency (MHz)')
plt.ylabel('dB')
plt.xlim(40)

The following 2 cells show that gain fluctations as a function of voltage are consistant well within uncertanty regardless of switch being on or off. 

In [ ]:
%matplotlib inline


plt.plot(freqs, df_gainData['13.30V_switchOff'], label = '13.3V')
plt.plot(freqs, df_gainData['14.55V_switchOff'], label = '14.55V')

plt.title('Switch Off')
plt.legend()
plt.xlabel('Frequency (MHz)')
plt.ylabel('Gain (dB)')
plt.xlim(40)

diff = df_gainData['14.55V_switchOff'] - df_gainData['13.30V_switchOff']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))


plt.figure()
plt.plot(freqs, diff)

plt.title('Switch off, 13.3V - 14.55V')

plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')
plt.xlim(40);

In [ ]:
%matplotlib inline


plt.plot(freqs, df_gainData['13.30V_switchOn'], label = '13.3V')
plt.plot(freqs, df_gainData['14.55V_switchOn'], label = '14.55V')

plt.title('Switch On')
plt.legend()
plt.xlabel('Frequency (MHz)')
plt.ylabel('Gain (dB)')
plt.xlim(40)

diff = df_gainData['14.55V_switchOn'] - df_gainData['13.30V_switchOn']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))

plt.figure()
plt.plot(freqs, df_gainData['14.55V_switchOn'] - df_gainData['13.30V_switchOn'])

plt.title('Switch On, 13.3V - 14.55V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')
plt.ylim(.14)
plt.xlim(40);

In [ ]:
%matplotlib inline

diff = df_gainData['14.20V_switchOn'] - df_gainData['13.30V_switchOn']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))

plt.figure()
plt.plot(freqs, diff)

plt.title('Switch On, 13.3V - 14.20V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')

In [ ]:
%matplotlib inline

diff = df_gainData['14.00V_switchOn'] - df_gainData['13.30V_switchOn']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))

plt.figure()
plt.plot(freqs, diff)

plt.title('Switch On, 13.3V - 14.00V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')

In [ ]:
%matplotlib inline

diff = df_gainData['13.50V_switchOn'] - df_gainData['13.30V_switchOn']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))

plt.figure()
plt.plot(freqs, diff)

plt.title('Switch On, 13.3V - 13.5V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')

In [ ]:
%matplotlib inline
#gain vs voltage plot

diffList = []
vListStr = ['14.55', '14.40', '14.20', '14.00', '13.50', '13.30', '13.00']
vList = [float(v) for v in vListStr]

for v in vListStr:
    diff = df_gainData[v+'V_switchOn'] - df_gainData['13.30V_switchOn']
    diffList.append(diff.mean())

def linDiffFit(V, m, b):
    diff = m*V + b
    return diff

popt, cov = opt.curve_fit(linDiffFit, vList, diffList)


print('fitting function: \ndiff = m*V + b\n')

print('fit parameters:')
print('m =', round(popt[0], 3), '+-', round(cov[0, 0], 5), 'dB/V')
print('b =', round(popt[1], 3),'+-', round(cov[1, 1], 5), 'dB')
vInterp = np.linspace(13, 14.6, 1000)

plt.scatter(vList, diffList)
plt.plot(vInterp, linDiffFit(vInterp, pop[0], pop[1]))
plt.xlabel('Voltage')
plt.ylabel('difference (dB)');

In [ ]:
%matplotlib inline
#gain vs voltage plot


def linDiffFit(V, m, b):
    diff = m*V + b
    return diff

def voltageExp(t, V0, tau, b):
    V = V0 * np.exp(-t/tau) + b
    return V

timeInterp = np.linspace(0, max(timeArr), 1000)
vInterp = voltageExp(timeInterp, popt[0], popt[1], popt[2])


popt1, cov = opt.curve_fit(linDiffFit, vList, diffList)
popt2, cov = opt.curve_fit(voltageExp, timeArrFull[:], vArrFull[:], p0=[1, 30, 13])


diffVsTime = linDiffFit(voltageExp(timeInterp, popt2[0], popt2[1], popt2[2]), 
                        popt1[0], popt1[1])



plt.plot(timeInterp, diffVsTime)
plt.xlabel('Time (minutes)')
plt.ylabel('Difference dB');
plt.xlim(0, 20)
plt.ylim(.05, .18)

plt.figure()
plt.plot(timeInterp, (10**(diffVsTime/10)- 1)* 100)
plt.xlabel('Time (minutes)')
plt.ylabel('Difference (linear percent)');
plt.xlim(0, 20)
plt.ylim(1.5, 4.2)

The following cells look at gain as switch is toggled

In [ ]:
%matplotlib inline


plt.plot(freqs, df_gainData['13.30V_switchOn'], label = 'On')
plt.plot(freqs, df_gainData['13.30V_switchOff'], label = 'Off')

plt.title('13.30V, Switch On vs Off')
plt.legend()
plt.xlabel('Frequency (MHz)')
plt.ylabel('Gain (dB)')
plt.xlim(40)

diff = df_gainData['13.30V_switchOn'] - df_gainData['13.30V_switchOff']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),4))

plt.figure()
plt.plot(freqs, diff)

plt.title('Difference, Switch On - Off, 13.3V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)');


In [ ]:
%matplotlib inline

diff = df_gainData['14.55V_switchOn'] - df_gainData['14.55V_switchOff']
print('difference mean =', round(diff.mean(),4))
print('difference std =', round(diff.std(),5))

plt.figure()
plt.plot(freqs, diff)

plt.title('Difference, Switch On - Off, 14.55V')
plt.xlabel('Frequency (MHz)')
plt.ylabel('Difference in gain (dB)')